# Log Viewer (Voila)

Dieses Dashboard liest eine Log-Datei aus demselben Ordner ein und visualisiert die Nutzung nach Zeit, Startmodus, Nutzer und Skript.

Erwartete Spalten im Log: `Date`, `Time`, `User`, `App`, `File`.

In [2]:
from pathlib import Path
from urllib.parse import unquote

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML

# Optional nicer charts; fallback to matplotlib if plotly is not installed.
try:
    import plotly.express as px
    HAS_PLOTLY = True
except Exception:
    HAS_PLOTLY = False
    import matplotlib.pyplot as plt


# ---------- Daten laden ----------
def find_log_file() -> Path:
    here = Path.cwd()
    default_file = here / "notebook_usage.log"
    if default_file.exists():
        return default_file

    candidates = sorted(here.glob("*.log"))
    if candidates:
        return candidates[0]

    raise FileNotFoundError(
        "Keine .log-Datei gefunden. Lege z. B. 'notebook_usage.log' in denselben Ordner wie dieses Notebook."
    )


def load_log_dataframe(log_path: Path) -> pd.DataFrame:
    df = pd.read_csv(log_path)

    required_cols = ["Date", "Time", "User", "App", "File"]
    missing = [col for col in required_cols if col not in df.columns]
    if missing:
        raise ValueError(f"Fehlende Spalten im Log: {missing}")

    df["File"] = df["File"].astype(str).map(unquote)
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df["Timestamp"] = pd.to_datetime(
        df["Date"].dt.strftime("%Y-%m-%d") + " " + df["Time"].astype(str),
        errors="coerce"
    )

    df = df.dropna(subset=["Date", "Timestamp"]).copy()
    df["Day"] = df["Date"].dt.date
    return df


log_path = find_log_file()
df_all = load_log_dataframe(log_path)


# ---------- Widgets ----------
min_day = df_all["Day"].min()
max_day = df_all["Day"].max()

start_picker = widgets.DatePicker(description="Von", value=min_day)
end_picker = widgets.DatePicker(description="Bis", value=max_day)

mode_options = ["Alle"] + sorted(df_all["App"].dropna().astype(str).unique().tolist())
mode_dropdown = widgets.Dropdown(options=mode_options, value="Alle", description="Modus")

user_options = sorted(df_all["User"].dropna().astype(str).unique().tolist())
user_select = widgets.SelectMultiple(
    options=user_options,
    value=tuple(user_options),
    rows=min(8, max(4, len(user_options))),
    description="User"
)

script_search = widgets.Text(
    value="",
    placeholder="Teil des Skriptnamens (optional)",
    description="Skript"
)

top_n = widgets.IntSlider(value=10, min=3, max=25, step=1, description="Top N")
user_sort = widgets.Dropdown(
    options=["Most starts", "User (A-Z)"],
    value="Most starts",
    description="User sort"
 )
refresh_btn = widgets.Button(description="Aktualisieren", button_style="primary")

out = widgets.Output()


# ---------- Filter + Darstellung ----------
def filtered_df() -> pd.DataFrame:
    data = df_all.copy()

    if start_picker.value is not None:
        data = data[data["Day"] >= start_picker.value]
    if end_picker.value is not None:
        data = data[data["Day"] <= end_picker.value]

    if mode_dropdown.value != "Alle":
        data = data[data["App"] == mode_dropdown.value]

    selected_users = list(user_select.value)
    if selected_users:
        data = data[data["User"].isin(selected_users)]

    q = script_search.value.strip().lower()
    if q:
        data = data[data["File"].str.lower().str.contains(q, na=False)]

    return data.sort_values("Timestamp")


def display_plotly_html(fig, include_js=False):
    include_plotlyjs = "cdn" if include_js else False
    html = fig.to_html(
        full_html=False,
        include_plotlyjs=include_plotlyjs,
        config={"responsive": True, "displaylogo": False}
    )
    display(HTML(html))


def show_plotly_charts(data: pd.DataFrame):
    daily = data.groupby("Day", as_index=False).size().rename(columns={"size": "Starts"})
    top_scripts = (
        data.groupby("File", as_index=False)
        .size()
        .rename(columns={"size": "Starts"})
        .sort_values("Starts", ascending=False)
        .head(top_n.value)
    )

    fig1 = px.line(daily, x="Day", y="Starts", markers=True, title="Starts pro Tag")
    fig1.update_layout(height=350)

    fig2 = px.bar(top_scripts, x="Starts", y="File", orientation="h", title=f"Top {top_n.value} Skripte")
    fig2.update_layout(height=420, yaxis={"categoryorder": "total ascending"})

    fig3 = px.pie(data, names="App", title="Anteil Startmodus")
    fig3.update_layout(height=350)

    # Voila/container-friendly rendering via HTML (wie im JV-Skript)
    display_plotly_html(fig1, include_js=True)
    display_plotly_html(fig2, include_js=False)
    display_plotly_html(fig3, include_js=False)


def show_matplotlib_charts(data: pd.DataFrame):
    daily = data.groupby("Day").size()
    top_scripts = data.groupby("File").size().sort_values(ascending=False).head(top_n.value)
    mode_counts = data.groupby("App").size().sort_values(ascending=False)

    fig, axes = plt.subplots(3, 1, figsize=(10, 13))

    daily.plot(ax=axes[0], marker="o")
    axes[0].set_title("Starts pro Tag")
    axes[0].set_xlabel("Tag")
    axes[0].set_ylabel("Starts")

    top_scripts.sort_values().plot(kind="barh", ax=axes[1])
    axes[1].set_title(f"Top {top_n.value} Skripte")
    axes[1].set_xlabel("Starts")

    mode_counts.plot(kind="pie", ax=axes[2], autopct="%1.1f%%")
    axes[2].set_title("Anteil Startmodus")
    axes[2].set_ylabel("")

    plt.tight_layout()
    display(fig)


def render(_=None):
    with out:
        out.clear_output(wait=True)

        try:
            data = filtered_df()
        except Exception as exc:
            display(HTML(f"<b style='color:#b91c1c;'>Fehler beim Filtern:</b> {exc}"))
            return

        if data.empty:
            display(HTML("<b>Keine Daten für die aktuelle Auswahl.</b>"))
            return

        total = len(data)
        unique_users = data["User"].nunique()
        unique_scripts = data["File"].nunique()

        display(HTML(
            f"""
            <h3>Überblick</h3>
            <ul>
              <li><b>Datei:</b> {log_path.name}</li>
              <li><b>Einträge:</b> {total}</li>
              <li><b>User:</b> {unique_users}</li>
              <li><b>Skripte:</b> {unique_scripts}</li>
              <li><b>Zeitraum:</b> {data['Day'].min()} bis {data['Day'].max()}</li>
            </ul>
            """
        ))

        top_users = (
            data.groupby("User", as_index=False)
            .size()
            .rename(columns={"size": "Starts"})
        )

        if user_sort.value == "Most starts":
            top_users = top_users.sort_values("Starts", ascending=False)
        else:
            top_users = top_users.sort_values("User", key=lambda s: s.astype(str).str.lower())

        top_users = top_users.head(15)

        display(HTML("<h4>User-Liste</h4>"))
        display(top_users)

        if HAS_PLOTLY:
            show_plotly_charts(data)
        else:
            show_matplotlib_charts(data)


for w in [start_picker, end_picker, mode_dropdown, user_select, script_search, top_n, user_sort]:
    w.observe(render, names="value")
refresh_btn.on_click(render)

controls = widgets.VBox([
    widgets.HTML(f"<b>Log-Datei:</b> {log_path}"),
    widgets.HBox([start_picker, end_picker, mode_dropdown, top_n]),
    widgets.HBox([user_select, script_search]),
    user_sort,
    refresh_btn,
])

display(controls, out)
render()

Output()